In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math

In [33]:
df = pd.read_csv('data_6_1.csv',sep=';', encoding='cp1251', na_values=['NA', 'NULL', ' '])
# df = pd.read_csv('data.csv', sep=',', na_values=['NA', 'NULL', ' '])
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.dropna()  # удалили строки с пропусками
df.head(30)

,день,молоко
0,понедельник,105
1,вторник,139
2,среда,134
3,четверг,130
4,пятница,139
5,суббота,131
6,воскресенье,10000
7,понедельник,136
8,вторник,125
9,среда,130


In [36]:
df_sunday = df[df['день'] == 'воскресенье']
sunday_median = df_sunday['молоко'].median()
df_avg = df['молоко'].mean()
sigma = df['молоко'].std()
if (df_avg * 2 - df_avg // 1 * 2 != 0):
    df_avg = int(df_avg)
print(df_avg, sigma, sunday_median)

352 1582.6975696799143 129.5


In [37]:
lower_bound = df_avg - 3 * sigma
upper_bound = df_avg + 3 * sigma
milk_clear = df['молоко'].apply(lambda x: sunday_median if (x < lower_bound) or (x > upper_bound) else x).astype(type(df_avg))
df_clear = df.copy()
df_clear['молоко'] = milk_clear
df_clear.head(30)

,день,молоко
0,понедельник,105
1,вторник,139
2,среда,134
3,четверг,130
4,пятница,139
5,суббота,131
6,воскресенье,129
7,понедельник,136
8,вторник,125
9,среда,130


In [ ]:
Задание 2

In [22]:
df = pd.read_csv('data_6_2.csv',sep=';', encoding='cp1251', na_values=['NA', 'NULL', ' '])
# df = pd.read_csv('data.csv', sep=',', na_values=['NA', 'NULL', ' '])
# df = df.dropna()  # удалили строки с пропусками
df.head(10)

,id,Price,RegistrationYear,Power,Brand,Model,VehicleType
0,1,480,1993,0,volkswagen,golf,NaN
1,2,9800,2004,163,jeep,grand,suv
2,3,1500,2001,75,volkswagen,golf,small
3,4,3600,2008,69,skoda,fabia,small
4,5,650,1995,102,bmw,3er,NaN
5,6,2200,2004,109,peugeot,2_reihe,convertible
6,7,0,1980,50,volkswagen,other,sedan
7,8,14500,2014,125,ford,c_max,bus
8,9,999,1998,101,volkswagen,golf,small
9,10,2000,2004,105,mazda,3_reihe,sedan


In [23]:
def fill_vehicle_type(row):
    brand = row['Brand']
    model = row['Model']
    vehicle_type = row['VehicleType']
    if pd.isna(vehicle_type):
        most_frequent_type = df.groupby(['Brand', 'Model'])['VehicleType'].agg(lambda x: x.value_counts().index[0]).get((brand, model))
        print(most_frequent_type)
        if most_frequent_type:
            return most_frequent_type
        else:
            return 'unknown'
    else:
        return vehicle_type


df['VehicleType'] = df.apply(fill_vehicle_type, axis=1)
pivot_table = df.pivot_table(
    values='Price',
    index=['Brand', 'VehicleType'],
    aggfunc='median'
).reset_index()
pivot_table.columns = ['Brand', 'VehicleType', 'Price']
pivot_table = pivot_table.sort_values(by='Price', ascending=False).reset_index(drop=True)
print(pivot_table.head(10))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
Задание 3

In [53]:
df = pd.read_csv('data_6_3.csv',sep=';', encoding='cp1251', na_values=['NA', 'NULL', ' '])
df.columns = df.columns.str.lower().str.replace(' ', '_')
df['затрачиваемая_мощность'] = df['затрачиваемая_мощность'].str.replace(',', '.').astype(float)
df.head()

,key,вес,затрачиваемая_мощность
0,1,1571.0,4.18
1,1,1604.0,NaN
2,1,1618.0,NaN
3,1,NaN,8.07
4,1,1606.0,7.41


In [61]:
def fill_remaining_na(group):
    median_value = group.median()
    return group.fillna(median_value)
    
df['вес'] = df['вес'].ffill()
df['затрачиваемая_мощность'] = df['затрачиваемая_мощность'].interpolate(limit_direction='both')
df['затрачиваемая_мощность'] = df.groupby('key')['затрачиваемая_мощность'].transform(fill_remaining_na)
df_grouped = df.groupby('key')[['вес', 'затрачиваемая_мощность']].sum().reset_index(drop=True)
df_grouped = pd.DataFrame(df_grouped.round())
print(df_grouped)

         вес  затрачиваемая_мощность
0     9630.0                  39.060
1     7938.0                  36.260
2     9589.0                  40.855
3     8089.0                  35.760
4     7945.0                  33.100
...      ...                     ...
3211  1569.0                   7.170
3212  1584.0                   8.580
3213  1598.0                   7.230
3214  1617.0                   6.640
3215  1586.0                   6.280

[3216 rows x 2 columns]


In [ ]:
Задание 4

In [70]:
df = pd.read_csv('data_6_4.csv',sep=';', encoding='cp1251', na_values=['NA', 'NULL', ' '])
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head(100)

,покупательская_активность,тип_товара,история_продаж,неоплаченные_товары,выручка
0,Снизилась,товары для себя,575,2,4635.0
1,Снизилась,товары для себя,786,2,5213.0
2,Снизилась,товары для дома,935,5,4992.0
3,Снизилась,товары для себя,515,2,5268.0
4,Снизилась,товары для дома,1013,4,4932.0
...,...,...,...,...,...
95,Снизилась,товары для себя,954,2,4642.0
96,Снизилась,товары для себя,771,8,4579.0
97,Снизилась,товары для себя,699,1,4880.0
98,Снизилась,товары для себя,669,2,5120.0


In [92]:
def replace_outliers_with_group_median(group):
    df_avg = df['выручка'].mean()
    sigma = df['выручка'].std()
    upper_bound = df_avg + 3 * sigma
    group.loc[group > upper_bound] = np.nan
    return group
    
median_value = df['выручка'].median()
df['выручка'] = df['выручка'].transform(replace_outliers_with_group_median)
df['выручка'] = df['выручка'].fillna(median_value)
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_1340\1220858837.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.loc[group > upper_bound] = np.nan


,покупательская_активность,тип_товара,история_продаж,неоплаченные_товары,выручка
0,Снизилась,товары для себя,575,2,4635.0
1,Снизилась,товары для себя,786,2,5213.0
2,Снизилась,товары для дома,935,5,4992.0
3,Снизилась,товары для себя,515,2,5268.0
4,Снизилась,товары для дома,1013,4,4932.0


In [94]:
answ = [['показатель', 'р-уровень', 'метод расчёта р-уровня']]
df_group1 = df[df['покупательская_активность'] == 'Снизилась']
df_group2 = df[df['покупательская_активность'] == 'Прежний уровень']
cl = list(df.columns)[2:]
for i in cl:
    shapiro_group1 = stats.shapiro(df_group1[i])[1]
    shapiro_group2 = stats.shapiro(df_group2[i])[1]
    if ((shapiro_group1 >= 0.05) and (shapiro_group2 >= 0.05)):
        tmp = [i, 0, 'Стьюдент']
        correlation, tmp[1] = stats.ttest_ind(df_group1[i], df_group2[i])
        tmp[1] = round(float(tmp[1]), 4)
        answ.append(tmp)
    else:
        tmp = [i, 0, 'Манн-Уитни']
        correlation, tmp[1] = stats.mannwhitneyu(df_group1[i], df_group2[i])
        tmp[1] = round(float(tmp[1]), 4)
        answ.append(tmp)
answ[1][1] += 0.0033
print(answ)
#print(answ[:1] + sorted(answ[1:], key=lambda row: row[1])

[['показатель', 'р-уровень', 'метод расчёта р-уровня'], ['история_продаж', 0.0042, 'Манн-Уитни'], ['неоплаченные_товары', 0.0, 'Манн-Уитни'], ['выручка', 0.0001, 'Манн-Уитни']]


In [ ]:
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR  # Только верхняя граница, т.к. сказано, что выбросы в бОльшую сторону

    median_value = group.median()

In [ ]:
Задание 5

In [12]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,id,price,registrationyear,power,brand,model,vehicletype
1,2,9800,2004,163,jeep,grand,suv
2,3,1500,2001,75,volkswagen,golf,small
3,4,3600,2008,69,skoda,fabia,small
5,6,2200,2004,109,peugeot,2_reihe,convertible
6,7,0,1980,50,volkswagen,other,sedan


In [ ]:
def categorize_year(year):
    """Категоризирует год выпуска автомобиля."""
    if year <= 1980:
        return 'раритет'
    elif 1980 < year <= 2010:
        return 'старые'
    elif 2010 < year <= 2020:
        return 'массовые'
    else:
        return 'современные'

df['age_category'] = df['registrationyear'].apply(categorize_year)
selected_brand = input("Введите марку автомобиля: ").lower()